# Création de votre premier mécanisme de recommandation de vidéos à la demande

Ce bloc-notes vous guide dans les étapes de la création d'un jeu de données Domain et d'un
mécanisme de recommandation qui renvoie des recommandations de films en fonction des données collectées à partir du jeu de données MovieLens. L'objectif est de recommander des films pertinents pour un utilisateur en particulier.

Les données proviennent du projet [MovieLens](https://grouplens.org/datasets/movielens/). Suivez le lien pour en savoir plus sur les données et les utilisations potentielles.

# Comment utiliser le bloc-notes

Le code est divisé en cellules comme celui donné ci-dessous. Il y a un bouton Exécuter triangulaire en haut de cette page qui vous permet d'exécuter chaque cellule et de passer à la suivante, ou vous pouvez appuyer sur `Shift` + `Enter` lorsque vous êtes dans la cellule pour l'exécuter et passer à la suivante.

Lorsqu'une cellule est en cours d'exécution, vous remarquerez une ligne sur le coté affichant une `*` pendant que la cellule est opérationnelle ou elle se mettra à jour sous forme de nombre pour indiquer la dernière cellule qui a terminé l'exécution après avoir achevé l'exécution de tous les codes dans une cellule.

Il suffit de suivre les instructions ci-dessous et d'exécuter les cellules pour commencer avec Amazon Personalize en utilisant des mécanismes de recommandation optimisés pour les cas.

## Importations
Python est livré avec une large gamme de bibliothèques. Nous devons les importer, ainsi que celles qui sont installées pour nous aider, comme [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK pour python) et [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), qui sont des outils essentiels pour la science des données.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

Ensuite, vous devez vérifier que votre environnement peut communiquer avec Amazon Personalize. Les lignes ci-dessous servent à cela.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Configurer les données
Les données sont importées dans Amazon Personalize par le biais d'Amazon S3. Nous allons spécifier ci-dessous un compartiment que vous avez créé dans AWS pour les besoins de cet exercice.
Vous allez mettre à jour la variable `bucket` ci-dessous pour qu'elle ait la valeur que vous avez créée plus tôt dans les étapes de CloudFormation, et qui doit se trouver dans un fichier texte de votre travail antérieur. Le `filename` n'a pas besoin d'être modifié.
### Spécifier un compartiment et un emplacement de sortie des données
Mettez à jour le nom `bucket` pour qu'il soit unique.

In [ ]:
filename = "movie-lens-100k.csv"

## Télécharger, préparer et charger les données d'entraînement
Actuellement, vous n'avez pas encore chargé les données de MovieLens localement pour les examiner. Exécutez les lignes ci-dessous pour télécharger la dernière copie et l'examiner rapidement.

### Télécharger et explorer le jeu de données

In [ ]:
!wget -N https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip

In [ ]:
!ls ml-latest-small

In [ ]:
!pygmentize ml-latest-small/README.txt

In [ ]:
interactions_data = pd.read_csv('./ml-latest-small/ratings.csv')
pd.set_option('display.max_rows', 5)
interactions_data

In [ ]:
interactions_data.info()

## Préparer les données

### Données d'interactions
Comme vous pouvez le voir, les données contiennent un UserID, un ItemID, un Rating et un Timestamp.

Nous allons maintenant retirer les articles ayant un faible classement, et supprimer la colonne Classement avant de créer notre modèle.

Nous ajoutons également la colonne EVENT_TYPE à toutes les interactions.

In [ ]:
interactions_data = interactions_data[interactions_data['rating'] > 3]                # Keep only movies rated higher than 3 out of 5.
interactions_data = interactions_data[['userId', 'movieId', 'timestamp']]
interactions_data.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True)
interactions_data['EVENT_TYPE']='watch' #Adding an EVENT_TYPE column that has the event type "watched" for all movies
interactions_data.head()

### Métadonnées des articles

Ouvrez le fichier de données des articles et examinez les premières lignes.

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)

In [ ]:
items_data.info()

In [ ]:
items_data['year'] = items_data['title'].str.extract('.*\((.*)\).*',expand = False)
items_data.head(5)

Sélection d'une date moderne comme horodatage de création pour cet exemple, car l'horodatage réel de création est inconnu. Dans votre cas d'utilisation, veuillez fournir l'horodatage de création approprié.

In [ ]:
ts= datetime.datetime(2022, 1, 1, 0, 0).strftime('%s')
print(ts)

In [ ]:
items_data["CREATION_TIMESTAMP"] = ts
items_data

In [ ]:
# removing the title
items_data.drop(columns="title", inplace = True)

# renaming the columns to match schema
items_data.rename(columns = { 'movieId':'ITEM_ID', 'genres':'GENRES',
                              'year':'YEAR'}, inplace = True)
items_data

# Métadonnées utilisateur

Le jeu de données n'ayant pas de métadonnées utilisateur, nous allons donc créer un faux champ de métadonnées.

In [ ]:
# get user ids from the interaction dataset

user_ids = interactions_data['USER_ID'].unique()
user_data = pd.DataFrame()
user_data["USER_ID"]=user_ids
user_data

## Ajout de métadonnées
Le jeu de données actuel ne contient pas d'informations supplémentaires sur les utilisateurs. Pour cet exemple, nous allons attribuer aléatoirement un sexe aux utilisateurs avec une probabilité égale homme et femme.

In [ ]:
possible_genders = ['female', 'male']
random = np.random.choice(possible_genders, len(user_data.index), p=[0.5, 0.5])
user_data["GENDER"] = random
user_data

## Configurer un compartiment S3 et un rôle IAM

Pour l'instant, nous avons téléchargé, manipulé et enregistré les données sur l'instance Amazon EBS associée à l'instance qui exécute ce bloc-notes Jupyter. Toutefois, Amazon Personalize aura besoin d'un compartiment S3 pour servir de source à vos données, ainsi que de rôles IAM pour accéder à ce compartiment. Mettons tout cela en place.

Le compartiment Amazon S3 doit se trouver dans la même région que les ressources Amazon Personalize que nous avons créées jusqu'à présent. Définissez simplement la région comme une chaîne ci-dessous.

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled this should be the same region as the Amazon Personalize resources
# region = "us-east-1"


In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedvod"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except:
    print("Bucket already exists. Using bucket", bucket_name)

### Charger les données vers S3
Maintenant que votre compartiment Amazon S3 a été créé, chargez le fichier CSV de nos données d'interaction utilisateur-article.

In [ ]:
interactions_filename = "interactions.csv"
interactions_data.to_csv(interactions_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

items_filename = "items.csv"
items_data.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

user_filename = "users.csv"
user_data.to_csv(user_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_filename).upload_file(user_filename)

## Définir la politique du compartiment S3
Amazon Personalize doit pouvoir lire le contenu de votre compartiment S3. Il faut donc ajouter une politique de compartiment qui l'autorise.

Remarque : Assurez-vous que le rôle que vous utilisez pour exécuter le code dans ce bloc-notes dispose des autorisations nécessaires pour modifier la politique du compartiment S3.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

## Créer et attendre un groupe de jeux de données
Le plus grand groupe dans Personalize est un groupe de jeux données qui isolera vos données, vos outils de suivi des événements, vos solutions, vos mécanismes de recommandations et vos campagnes. Regroupement de choses qui partagent un ensemble commun de données. N'hésitez pas à modifier le nom ci-dessous si vous le souhaitez.

### Créer un groupe de jeux de données

In [ ]:
response = personalize.create_dataset_group(
    name='personalize-video-on-demand-ds-group',
    domain='VIDEO_ON_DEMAND'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

Attendre que le groupe de jeu de données ait le statut ACTIF
Avant de pouvoir utiliser le groupe de jeu de données dans les articles ci-dessous, il doit être actif; exécutez la cellule ci-dessous et attendez qu'il soit actif.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Créer un schéma d'interactions
L'un des principaux articles permettant à Personalize de comprendre vos données provient du schéma défini ci-dessous. Cette configuration indique au service comment traiter les données fournies par votre fichier CSV. Notez que les colonnes et les types s'alignent sur les articles présents dans le fichier que vous avez créé ci-dessus.

In [ ]:
schema = {
  "type": "record",
  "name": "Interactions",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "EVENT_TYPE",
          "type": "string"
      },
      {
          "name": "TIMESTAMP",
          "type": "long"
      }
  ],
  "version": "1.0"
}

create_interactions_schema_response = personalize.create_schema(
    name='personalize-demo-interactions-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

interactions_schema_arn = create_interactions_schema_response['schemaArn']
print(json.dumps(create_interactions_schema_response, indent=2))

# Créer un schéma d'articles (films)

In [ ]:
schema = {
  "type": "record",
  "name": "Items",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
    {
      "name": "ITEM_ID",
      "type": "string"
    },
    {
      "name": "GENRES",
      "type": [
        "string"
      ],
      "categorical": True
    },
    {
      "name": "YEAR",
      "type": [
        "string"
      ],
      "categorical": True
    }, 
    {
      "name": "CREATION_TIMESTAMP",
      "type": "long"
    }
  ],
  "version": "1.0"
}
create_items_schema_response = personalize.create_schema(
    name='personalize-demo-items-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

items_schema_arn = create_items_schema_response['schemaArn']
print(json.dumps(create_items_schema_response, indent=2))

# Créer un schéma des utilisateurs

In [ ]:
schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "GENDER",
          "type": "string",
          "categorical": True
      }
    ],
    "version": "1.0"
}
create_users_schema_response = personalize.create_schema(
    name='personalize-demo-users-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

users_schema_arn = create_users_schema_response['schemaArn']
print(json.dumps(create_users_schema_response, indent=2))

## Créer des jeux de données
Après avoir créé le groupe, vous devez créer les jeux de données proprement dits.

### Créer un jeu de données d'interactions

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Créer un jeu de données d'articles

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Créer un jeu de données d'utilisateurs

In [ ]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-users",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Créer un rôle Personalize
De plus, Amazon Personalize doit pouvoir assumer des rôles dans AWS, afin d'avoir les autorisations d'exécuter certaines tâches. Les lignes ci-dessous les accordent.

Remarque : Assurez-vous que le rôle que vous utilisez pour exécuter le code dans ce bloc-notes dispose des autorisations nécessaires pour créer un rôle.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleVODDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## Importer les données
Vous avez précédemment créé le groupe de jeu de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données de S3 dans Amazon Personalize pour les utiliser dans la création de votre modèle.
### Créer une tâche d'importation de jeux de données d'interactions

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-interactions",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

### Créer une tâche d'importation de jeux de données d'articles

In [ ]:
create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-items",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_filename)
    },
    roleArn = role_arn
)

dataset_items_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))

### Créer une tâche d'importation de jeux de données d'utilisateurs

In [ ]:
create_users_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-users",
    datasetArn = users_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_filename)
    },
    roleArn = role_arn
)

dataset_users_import_job_arn = create_users_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_users_dataset_import_job_response, indent=2))

Attendre que la tâche d'importation de données ait le statut ACTIF
La tâche d'importation peut durer un certain temps ; veuillez patienter jusqu'à ce que vous voyiez qu'elle est active ci-dessous.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Interactions DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_items_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Items DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_users_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Users DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Choisir des cas d'utilisation d'un mécanisme de recommandation

Chaque domaine a des cas d'utilisation différents. Lorsque vous créez un mécanisme de recommandation, vous le créez pour un cas d'utilisation spécifique, et chaque cas d'utilisation a des exigences différentes pour obtenir des recommandations.


In [ ]:
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND') # See a list of recommenders for the domain. 
if (len(available_recipes["recipes"])==0):
    # This is a workaround to get the recipes in case 'available_recipes["recipes"]'does not retrieve them
    available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND', nextToken=available_recipes["nextToken"])
display(available_recipes["recipes"])
    

Nous allons créer un mécanisme de recommandation du type "Davantage comme X". Ce type de mécanisme de recommandation propose des recommandations de vidéos similaires à une vidéo qu'a regardé un utilisateur. Avec ce cas d'utilisation, Amazon Personalize filtre automatiquement les vidéos que l'utilisateur a regardées en fonction du userId spécifié dans l'appel `get_recommendations`. Pour améliorer les performances, enregistrez les événements Click en plus des événements Watch requis.

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'more_like_x_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-more-like-x',
  datasetGroupArn = dataset_group_arn
)
recommender_more_like_x_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Nous allons créer un deuxième mécanisme de recommandation du type "Meilleurs choix pour vous". Ce type de mécanisme de recommandation offre des recommandations personnalisées de contenu en continu pour un utilisateur que vous spécifiez. Avec ce cas d'utilisation, Amazon Personalize filtre automatiquement les vidéos que l'utilisateur a regardées en fonction du userId que vous spécifiez et des événements `Watch`.

[Plus de cas d'utilisation par domaine](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'top_picks_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-top-picks',
  datasetGroupArn = dataset_group_arn
)
recommender_top_picks_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Nous attendons que les mécanismes de recommandations aient fini de créer et qu'ils aient le statut `ACTIVE`. Nous vérifions régulièrement l'état du mécanisme de recommandation

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_more_like_x_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_more_like_x_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_more_like_x_arn))

    if status == "ACTIVE":
        break
    else:
        print("The More Like X Recommender build is still in progress")
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_top_picks_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_top_picks_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_top_picks_arn))

    if status == "ACTIVE":
        break
    else:
        print("The Top Pics for You Recommender build is still in progress")
        
    time.sleep(60)

# Obtenir des recommandations avec un mécanisme de recommandation
Maintenant que les mécanismes de recommandation ont été entraînés, examinons les recommandations que nous pouvons obtenir pour nos utilisateurs.

In [ ]:
# reading the original data in order to have a dataframe that has both movie_ids 
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./ml-latest-small/movies.csv')
items_df.sample(10)

In [ ]:
def get_movie_by_id(movie_id, movie_df):
    """
    This takes in an movie_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[movie_df["movieId"]==int(movie_id)]['title'].values[0]
    except:
        print (movie_id)
        return "Error obtaining title"

### Demandons des recommandations « Davantage comme X » :

In [ ]:
# First pick a user
test_user_id = "1"

# Select a random item
test_item_id = "81847" #Iron Man 59315, Tangled: 81847

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_more_like_x_arn,
    userId = test_user_id,
    itemId = test_item_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_movie_by_id(test_item_id, items_df)])

pd.options.display.max_rows = 20
display(user_recommendations_df)

### Obtenez des recommandations du mécanisme de recommandation qui retourne les meilleures sélections ("Meilleurs choix pour vous") :

En ajoutant les métadonnées de l'utilisateur à notre exemple d'utilisateur, vous pouvez utiliser ce type de métadonnées pour obtenir des informations sur vos utilisateurs.

In [ ]:
users_data_df = pd.read_csv('./users.csv')

def get_gender_by_id(user_id, user_df):
    """
    This takes in a user_id and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    try:
        return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    except:
        print (user_id)
        return "Error obtaining title"

In [ ]:
# First pick a user
test_user_id = "111" # samples users: 55, 75, 76, 111

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_top_picks_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

column_name = test_user_id+" ("+get_gender_by_id(test_user_id, users_data_df)+")"

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [column_name])

pd.options.display.max_rows =20
display(user_recommendations_df)

## Examiner
En utilisant les codes ci-dessus, vous avez entraîné un modèle de deep learning pour générer des recommandations de films en fonction du comportement antérieur de l'utilisateur. Vous avez créé deux mécanismes de recommandation pour deux cas d'utilisation fondamentaux. 
À l'avenir, vous pourrez adapter ce code pour créer d'autres mécanismes de recommandations.

## Remarques pour le bloc-notes suivant :
Vous aurez besoin de quelques valeurs pour le bloc-notes suivant. Exécutez la cellule ci-dessous pour les stocker, afin qu'elles puissent être utilisées dans le bloc-notes `Clean_Up_Resources.ipynb`.

Toutes les données stockées pour ces variables seront remplacées et définies aux valeurs spécifiées dans ce bloc-notes. 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region

Si vous avez exécuté le bloc-notes `Building_Your_First_Recommender_Ecommerce.ipynb`, veillez à réexécuter l'étape précédente du bloc-notes `Building_Your_First_Recommender_Ecommerce.ipynb` et `Clean_Up_Resources.ipynb` pour supprimer les ressources créées dans ce bloc-notes après avoir exécuté `Clean_Up_Resources.ipynb` avec les ressources créées ici.